In [1]:
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data
import os
from acquire import get_titanic_data
from prepare import clean_titanic
from env import host,db,protocol,password,user,mysqlcon

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
import acquire
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.preprocessing import LabelEncoder

### Lesson on Regression & Regression Exercise

In [2]:
filename = "student_grades.csv"
os.path.isfile(filename)

True

In [3]:
def aquire_student_grades():
    filename = "student_grades.csv"

    mysqlcon=f"{protocol}://{user}:{password}@{host}/{db}"

    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        
      # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)
        return df
    
  


In [4]:
df=aquire_student_grades()

In [5]:
df = pd.read_csv(filename)

In [6]:
df = df.drop(columns='student_id')

In [7]:
df.isna().sum()

Unnamed: 0.18    0
Unnamed: 0.17    0
Unnamed: 0.16    0
Unnamed: 0.15    0
Unnamed: 0.14    0
Unnamed: 0.13    0
Unnamed: 0.12    0
Unnamed: 0.11    0
Unnamed: 0.10    0
Unnamed: 0.9     0
Unnamed: 0.8     0
Unnamed: 0.7     0
Unnamed: 0.6     0
Unnamed: 0.5     0
Unnamed: 0.4     0
Unnamed: 0.3     0
Unnamed: 0.2     0
Unnamed: 0.1     0
Unnamed: 0       0
exam1            1
exam2            0
exam3            1
final_grade      0
dtype: int64

In [8]:
df['exam3'].isna().value_counts()

False    103
True       1
Name: exam3, dtype: int64

In [9]:
df['exam1'].isna().value_counts()

False    103
True       1
Name: exam1, dtype: int64

In [10]:
df.dropna().shape

(102, 23)

In [11]:
df = df.dropna()

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 103
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0.18  102 non-null    int64  
 1   Unnamed: 0.17  102 non-null    int64  
 2   Unnamed: 0.16  102 non-null    int64  
 3   Unnamed: 0.15  102 non-null    int64  
 4   Unnamed: 0.14  102 non-null    int64  
 5   Unnamed: 0.13  102 non-null    int64  
 6   Unnamed: 0.12  102 non-null    int64  
 7   Unnamed: 0.11  102 non-null    int64  
 8   Unnamed: 0.10  102 non-null    int64  
 9   Unnamed: 0.9   102 non-null    int64  
 10  Unnamed: 0.8   102 non-null    int64  
 11  Unnamed: 0.7   102 non-null    int64  
 12  Unnamed: 0.6   102 non-null    int64  
 13  Unnamed: 0.5   102 non-null    int64  
 14  Unnamed: 0.4   102 non-null    int64  
 15  Unnamed: 0.3   102 non-null    int64  
 16  Unnamed: 0.2   102 non-null    int64  
 17  Unnamed: 0.1   102 non-null    int64  
 18  Unnamed: 0

In [13]:
df['exam1'].value_counts()

70.0     16
100.0     8
98.0      8
83.0      8
93.0      8
79.0      8
92.0      8
62.0      8
58.0      8
57.0      8
85.0      7
73.0      7
Name: exam1, dtype: int64

In [14]:
df['exam1'] = df.exam1.astype(int)
df['exam3'] = df.exam3.astype(int)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 103
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Unnamed: 0.18  102 non-null    int64
 1   Unnamed: 0.17  102 non-null    int64
 2   Unnamed: 0.16  102 non-null    int64
 3   Unnamed: 0.15  102 non-null    int64
 4   Unnamed: 0.14  102 non-null    int64
 5   Unnamed: 0.13  102 non-null    int64
 6   Unnamed: 0.12  102 non-null    int64
 7   Unnamed: 0.11  102 non-null    int64
 8   Unnamed: 0.10  102 non-null    int64
 9   Unnamed: 0.9   102 non-null    int64
 10  Unnamed: 0.8   102 non-null    int64
 11  Unnamed: 0.7   102 non-null    int64
 12  Unnamed: 0.6   102 non-null    int64
 13  Unnamed: 0.5   102 non-null    int64
 14  Unnamed: 0.4   102 non-null    int64
 15  Unnamed: 0.3   102 non-null    int64
 16  Unnamed: 0.2   102 non-null    int64
 17  Unnamed: 0.1   102 non-null    int64
 18  Unnamed: 0     102 non-null    int64
 19  exam1   

In [16]:
def clean_student_grades(df):
    '''
    clean_student_grades(df) takes in a single pandas dataframe
    removed the student id columns, removes records with null values, and cast any floats
    into integers.Returns a clead df.
    '''
    #drops student_id column
    df = df.drop(columns='student_id')
    #drops na values 
    df = df.dropna()
    #turn whole df into a integer. float values match int values
    df = df.astype(int)
    return df

In [17]:
def split_my_students(df):
    '''
    split_my_students(df) takes in a a clean dataframe, referencing the cleaned
    version of student data. Split the data into train, validate, test
    
    pass df as an arguement in the function 
    returns: train, validate, test: three pandas dataframes
    '''
    train_val, test = train_test_split(df, random_state=1349, train_size=0.7)
    
    train, validate = train_test_split(train_val, random_state=1349, train_size=.10)
    
    return train, validate, test

In [18]:
def wrangle_grades():
    '''
    wrangle_grades does the split and clean and returns the data'''
    return split_my_students(
        clean_student_grades(
            aquire_student_grades()))

In [19]:
train, validate, test = wrangle_grades()

In [20]:
train.shape, validate.shape, test.shape

((7, 23), (64, 23), (31, 23))

### Exercise Regression Zillow

In [21]:
def get_zillow_data():
    filename = "zillow_exc.csv"
    mysqlcon=f"{protocol}://{user}:{password}@{host}/zillow"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read the SQL query into a dataframe
        df = pd.read_sql_query('''select bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, 
                                taxvaluedollarcnt, yearbuilt, taxamount, fips, propertylandusetypeid,
                                propertylandusedesc
                                from properties_2017
                                left join propertylandusetype
                                using (propertylandusetypeid)''', mysqlcon)

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)

        # Return the dataframe to the calling code
        return df


In [22]:
df = get_zillow_data()

In [23]:
df

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid,propertylandusedesc
0,0,0.0,0.0,NaN,9.0,NaN,NaN,6037.0,269.0,Planned Unit Development
1,1,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,261.0,Single Family Residential
2,2,0.0,0.0,73026.0,1434941.0,1959.0,20800.37,6037.0,47.0,Store/Office (Mixed Use)
3,3,0.0,0.0,5068.0,1174475.0,1948.0,14557.57,6037.0,47.0,Store/Office (Mixed Use)
4,4,0.0,0.0,1776.0,440101.0,1947.0,5725.17,6037.0,31.0,Commercial/Office/Residential Mixed Used
...,...,...,...,...,...,...,...,...,...,...
2985212,2985212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985213,2985213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985214,2985214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985215,2985215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#Replace a whitespace sequence or empty with a NaN value and reassign this manipulation to df.
df = df.replace(r'^\s*$', np.nan, regex=True)

In [26]:
df

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid,propertylandusedesc
0,0,0.0,0.0,NaN,9.0,NaN,NaN,6037.0,269.0,Planned Unit Development
1,1,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,261.0,Single Family Residential
2,2,0.0,0.0,73026.0,1434941.0,1959.0,20800.37,6037.0,47.0,Store/Office (Mixed Use)
3,3,0.0,0.0,5068.0,1174475.0,1948.0,14557.57,6037.0,47.0,Store/Office (Mixed Use)
4,4,0.0,0.0,1776.0,440101.0,1947.0,5725.17,6037.0,31.0,Commercial/Office/Residential Mixed Used
...,...,...,...,...,...,...,...,...,...,...
2985212,2985212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985213,2985213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985214,2985214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985215,2985215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df = df[df['propertylandusedesc']=='Single Family Residential']

In [28]:
df[df['propertylandusedesc']=='Inferred Single Family Residential']

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid,propertylandusedesc


In [29]:
df

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid,propertylandusedesc
1,1,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,261.0,Single Family Residential
15,15,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0,Single Family Residential
16,16,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0,Single Family Residential
17,17,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0,261.0,Single Family Residential
20,20,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,261.0,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...
2982268,2982268,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0,261.0,Single Family Residential
2982270,2982270,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0,261.0,Single Family Residential
2982272,2982272,0.0,0.0,NaN,208057.0,NaN,5783.88,6059.0,261.0,Single Family Residential
2982275,2982275,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0,261.0,Single Family Residential


In [30]:
#checked for missing values in targeted columns
df.isna().value_counts()

Unnamed: 0  bedroomcnt  bathroomcnt  calculatedfinishedsquarefeet  taxvaluedollarcnt  yearbuilt  taxamount  fips   propertylandusetypeid  propertylandusedesc
False       False       False        False                         False              False      False      False  False                  False                  2140235
                                     True                          False              True       False      False  False                  False                     6154
                                     False                         False              False      True       False  False                  False                     2568
                                                                                      True       False      False  False                  False                     1439
                                     True                          False              True       True       False  False                  False                     13

In [31]:
#filled nan values with 0 
df = df.fillna(0)

In [32]:
def wrangle_zillow(df):
    '''Wrangle zillow will clean the data and update missing values 
    for the df'''
   #Replace a whitespace sequence or empty with a NaN value and reassign this manipulation to df.
    df = df.replace(r'^\s*$', np.nan, regex=True)    
    #drop any duplicates in the df
    df = df.drop_duplicates()
    #drop redundant column
    df = df.drop(columns = 'propertylandusedesc')
    #filled nan values with 0 
    df = df.fillna(0)
    return df


In [33]:
df = wrangle_zillow(df)

In [34]:
df[df['calculatedfinishedsquarefeet']==0]

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid
1,1,0.0,0.0,0.0,27516.0,0.0,0.00,6037.0,261.0
15,15,0.0,0.0,0.0,10.0,0.0,0.00,6037.0,261.0
16,16,0.0,0.0,0.0,10.0,0.0,0.00,6037.0,261.0
17,17,0.0,0.0,0.0,2108.0,0.0,174.21,6037.0,261.0
27,27,0.0,0.0,0.0,124.0,0.0,0.00,6037.0,261.0
...,...,...,...,...,...,...,...,...,...
2982202,2982202,0.0,0.0,0.0,2568893.0,0.0,27309.30,6059.0,261.0
2982239,2982239,0.0,0.0,0.0,92679.0,0.0,1090.16,6111.0,261.0
2982262,2982262,0.0,0.0,0.0,1198476.0,0.0,0.00,6037.0,261.0
2982267,2982267,0.0,0.0,0.0,1087111.0,0.0,19313.08,6059.0,261.0


In [38]:
df.isna().mean()

Unnamed: 0                      0.0
bedroomcnt                      0.0
bathroomcnt                     0.0
calculatedfinishedsquarefeet    0.0
taxvaluedollarcnt               0.0
yearbuilt                       0.0
taxamount                       0.0
fips                            0.0
propertylandusetypeid           0.0
dtype: float64